In [2]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [3]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-30 19:22:54 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-30 19:22:54 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-30 19:22:54 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-30 19:22:54 [INFO]: Loaded successfully!


<h1>Training data</h1>

<h3>Loading training database</h3>

In [4]:
train_X = physionet2012_dataset['train_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [5]:
distribution_gender_training = train_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_training # 0: female, or 1: male

Gender
 1.0    55.338287
 0.0    44.518316
-1.0     0.143397
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [6]:
distribution_ICUType_training = train_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_training

ICUType
3.0    35.862339
4.0    28.549081
2.0    21.040282
1.0    14.548299
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [7]:
age_uniques_train = train_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_train.head()

,RecordID,Age
0,132539,54.0
1,132541,44.0
2,132547,64.0
3,132548,68.0
4,132568,66.0


<h3>Showing total occurrences of age</h3>

In [8]:
# 60% de 11988 (training set)
age_uniques_train.count()

RecordID    7671
Age         7671
dtype: int64

In [8]:
#first_values = train_X['Age'][::48] #first value of "Age" at every 48th index
#total_quantity_of_age_occurrences = first_values.count()
#total_quantity_of_age_occurrences

In [9]:
#first_values.head()

<h3>Dividing age into subgroup of people aged 65+</h3>

In [9]:
more_than_or_equal_to_65_training = age_uniques_train[age_uniques_train['Age'] >= 65]
# more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
#distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
#distribution_age_group_one
# age_uniques_more_than_or_equal_to_65 = more_than_or_equal_to_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_one = (more_than_or_equal_to_65_training.count() / age_uniques_train.count())*100
percentage_group_one

RecordID    54.803807
Age         54.803807
dtype: float64

<h3>Dividing age into subgroup of people under 65 years of age</h3>

In [10]:
less_than_65 = age_uniques_train[age_uniques_train['Age'] < 65]
# less_than_65 = train_X[train_X['Age'] < 65] 
#distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
#distribution_age_group_two

# age_uniques_less_than_65 = less_than_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_two = (less_than_65.count() / age_uniques_train.count()) * 100
percentage_group_two

RecordID    45.196193
Age         45.196193
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [11]:
filtered_train_X = train_X[(train_X['Height'] != -1) & (train_X['Weight'] != -1) & (train_X['Height'].notna()) & (train_X['Weight'].notna())] 

In [12]:
height_uniques_train = filtered_train_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_train.head()

,RecordID,Height
0,132547,180.3
1,132548,162.6
2,132568,157.5
3,132570,170.2
4,132573,162.6


In [13]:
weight_uniques_train = filtered_train_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_train.head()

,RecordID,Weight
0,132547,114.0
1,132548,87.0
2,132568,84.5
3,132570,102.6
4,132573,90.1


<h3>Calculate the BMI</h3>

In [15]:
def classify_BMI(BMI):
    if BMI < 18.5:
        return "Baixo peso"
    elif BMI >= 18.6 and BMI <= 24.9:
        return "Peso normal"
    elif BMI >= 25 and BMI <= 29.9:
        return "Sobrepeso"
    elif BMI >= 30 and BMI <= 34.9:
        return "Obesidade grau 1"
    elif BMI >= 35 and BMI <= 39.9:
        return "Obesidade grau 2"
    elif BMI >= 40:
        return "Obesidade grau 3"

In [16]:
height_uniques_train['Height'] = height_uniques_train['Height'] / 100 # Converting Height from cm to meters

In [17]:
bmi_data = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data["RecordID"] = height_uniques_train["RecordID"]
bmi_data["Height"] = height_uniques_train["Height"]
bmi_data["Weight"] = weight_uniques_train["Weight"]
bmi_data["BMI"] = weight_uniques_train["Weight"] / (height_uniques_train["Height"] ** 2)
bmi_data["Classificacao"] = bmi_data["BMI"].apply(classify_BMI)
bmi_data.head()
# Creating a new DataFrame to store relevant data
#bmi_data = filtered_train_X[['RecordID', 'Height', 'Weight', 'BMI']].reset_index(drop=True)

#bmi_data = pd.concat([height_uniques_train['RecordID'], height_uniques_train['Height'], weight_uniques_train['Weight']], sort=True)
#bmi_data = pd.merge(height_uniques_train[['RecordID', 'Height']], weight_uniques_train[['RecordID', 'Weight']], on='RecordID')
#bmi_data.head()
# bmi_data['BMI'] = weight_uniques_train['Weight'] / (height_uniques_train['Height'] ** 2)


,RecordID,Height,Weight,BMI,Classificacao
0,132547,1.803,114.0,35.068194,Obesidade grau 2
1,132548,1.626,87.0,32.906233,Obesidade grau 1
2,132568,1.575,84.5,34.063996,Obesidade grau 1
3,132570,1.702,102.6,35.418344,Obesidade grau 2
4,132573,1.626,90.1,34.078754,Obesidade grau 1


<h3>Percentage of BMI classification groups</h3>

In [19]:
bmi_data["Classificacao"].value_counts(normalize=True) * 100

Classificacao
Sobrepeso           34.566038
Peso normal         29.710692
Obesidade grau 1    18.465409
Obesidade grau 2     7.572327
Obesidade grau 3     6.943396
Baixo peso           2.742138
Name: proportion, dtype: float64

<h1>Validation data</h1>

In [11]:
validation_X = physionet2012_dataset['val_X']

In [12]:
age_uniques_validation = validation_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_validation.count()

RecordID    1918
Age         1918
dtype: int64

<h1>Test data</h1>

In [14]:
test_X = physionet2012_dataset['test_X']

In [15]:
age_uniques_test = test_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_test.count()

RecordID    2399
Age         2399
dtype: int64